In [1]:
from configparser import ConfigParser
import datetime
import pandas as pd
import pyodbc

In [2]:
try:
    cd /notebooks/notebooks
except:
    %cd /notebooks/notebooks


/notebooks/notebooks


In [3]:

CurrentDate = str(datetime.datetime.now())

delimiter1 = " ";
delimiter2 = "-";
CurrentDate = CurrentDate.replace(delimiter2, delimiter1);
CD = CurrentDate.split(delimiter1);
date = CD[2]
month = CD[1]
year = CD[0]


In [4]:
year

'2023'

In [13]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [14]:
count_of_audit_ingt = pd.read_sql_query("select count(1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN where JOB_NAME = 'RISK_ORGANIZATION' and JOB_STATUS = 'Success'",conn)

In [15]:
count_of_audit_ingt = count_of_audit_ingt.iloc[0,0]

In [16]:
count_of_audit_ingt

31

In [18]:
try:
    
    if ((date == '25' and month == '05') or ((date == '01' and month == '05')) or (count_of_audit_ingt == 0)):

#         conn.autocommit = False
        sql_delete_query1  = "delete from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION"
        sql_delete_query2  = "delete from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_ANALYTICAL_RULES"
        sql_delete_query3  = "delete from FINCORE_DB.Fincore_Analytics.RISK_LKP_ENTITY_THRESHOLD where RULE_NO like 'O0%'"
        sql_delete_query4  = "delete from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES"
        sql_delete_query5  = "delete from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG"
        sql_delete_query6  = "delete from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL"
        sql_delete_query7  = "delete from FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_PEER_GRP_ATTRIBUTES WHERE ENTITY_TYPE = 'O' "

        
        cursor.execute(sql_delete_query1)
        cursor.execute(sql_delete_query2)
        cursor.execute(sql_delete_query3)
        cursor.execute(sql_delete_query4)
        cursor.execute(sql_delete_query5)
        cursor.execute(sql_delete_query6)
        cursor.execute(sql_delete_query7)

        conn.commit()
        
        
        %run Organization_Risk/Master_data_pop_all_data_ORG.ipynb
        %run Organization_Risk/PEER_GROUPING_ORG.ipynb
        %run Organization_Risk/Threshold_Computation_V2_ORG.ipynb
        %run Organization_Risk/Entity_Risk_Calculation_V2_1_org.ipynb
        %run Organization_Risk/Entity_calculation_2_CR_ORG.ipynb
        

    else:
        print("else")
        %run Organization_Risk/master_data_pop_incremental_ORG.ipynb
        %run Organization_Risk/Entity_Risk_V2_Incremental_data_ORG.ipynb
        %run Organization_Risk/Threshold_Computation_V2_Incremental_data_ORG.ipynb
        %run Organization_Risk/Entity_Risk_Calculation_V2_1_Incremental_data_CR_ORG.ipynb
        
        


        
        
except Exception as error:
    config = ConfigParser()
    config.read(r'/notebooks/notebooks/config.ini')
    connection_string = config.get('SERVER', 'connection')
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    
    %store -r job_id
    error = str(error)
    error = error.replace('\'','')
    error = error.replace('\"','')
    print(error)
    print(f'''update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set ERROR_DESC = '{error}'  where  JOB_RUN_ID = {job_id}''')
    conn.execute(f'''update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set ERROR_DESC = '{error}'  where  JOB_RUN_ID = {job_id}''')
    conn.commit()
    raise SystemExit
    conn.close()
    

    

    
    
    
    


Stored 'job_id' (int64)
master_data_pop_org_q1  DONE!
master_data_pop_org_q2  DONE!
org_pincode_update  DONE!
org_occ_update  DONE!
master_data_pop_per_income  DONE!
tran_amnt  DONE!
peer_grouping_columns  DONE!
MOBILE_COUNT  DONE!
TELEPHONE_COUNT  DONE!
EMAIL_COUNT  DONE!
WATCHLIST_COUNT  DONE!
STR_REPORT_COUNT  DONE!
CTR_REPORT_COUNT  DONE!
NTR_REPORT_COUNT  DONE!
PTR_COUNT  DONE!
CBWTR_REPORT_COUNT  DONE!
CCR_REPORT_COUNT  DONE!
str_amount_debit  DONE!
str_amount_credit  DONE!
ctr_amount_debit  DONE!
ctr_amount_credit  DONE!
ntr_amount_debit  DONE!
ntr_amount_credit  DONE!
cbwtr_amount_debit  DONE!
cbwtr_amount_credit  DONE!
ccr_amount_debit  DONE!
ccr_amount_credit  DONE!
('MERGE INTO '
 'FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES      WITH '
 '(HOLDLOCK) AS TARGET USING (   \n'
 'SELECT PM.ORGANIZATION_MASTER_ID , \n'
 "'O001001000' AS RULE_NO,  \n"
 'WL.AGENCY_NAME AS RULE_VALUE,  \n'
 '10 AS RISK_SCORE_STATIC ,   \n'
 'GETDATE() AS CREATE_DATE   \n'
 'FROM 

('\r\n'
 '                    MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES \r\n'
 '\t\t\t\t\tWITH (HOLDLOCK) as target USING       \r\n'
 '\t\t\t\t\t(      \r\n'
 '\t\t\t\t\tSELECT   PMAM.ORGANIZATION_Master_Id,    \r\n'
 '\t\t\t\t\tCOUNT(DISTINCT AM.Account_Master_Id) AS COUNT_OF_WALLET_CARDS  \r\n'
 '\t\t\t\t\tFROM [FINCORE_DB].[Fincore].Account_Master AM  with (nolock)\r\n'
 '\t\t\t\t\tinner JOIN '
 '[FINCORE_DB].[Fincore].ORGANIZATION_Master_Account_Master PMAM  with '
 '(nolock)    \r\n'
 '\t\t\t\t\tON AM.Account_Master_Id = PMAM.Account_Master_Id inner join \r\n'
 '\t\t\t\t\tFINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL k '
 'with(nolock) \r\n'
 '\t\t\t\t\ton PMAM.ORGANIZATION_Master_Id=k.organization_master_id\r\n'
 '\t\t\t\t\t\r\n'
 "\t\t\t\t\tWHERE PMAM.Is_Active_Flag = 1 AND AM.Account_Type in ( 'Prepaid "
 "card','gift card','Wallet card')\r\n"
 '\t\t\t\t\tGROUP BY PMAM.ORGANIZATION_Master_Id)       \r\n'
 '\t\t\t\t\tAS source 

Done
('\r\n'
 '                        MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES   \r\n'
 '\t\t\t\t\t\tWITH (HOLDLOCK) as target USING        \r\n'
 '\t\t\t\t\t\t(          \r\n'
 '\t\t\t\t\t\tselect  PMAM.ORGANIZATION_Master_Id ,        \r\n'
 '\t\t\t\t\t\tCount(Distinct PMAM.Account_Master_Id) as No_Of_Accounts,   \r\n'
 "\t\t\t\t\t\tCount(Distinct ( cast (am.Re_Id as varchar) +'|'+ am.Branch_code "
 ') ) as No_Of_Banks     \r\n'
 '\t\t\t\t\t\tFROM [FINCORE_DB].[Fincore].ORGANIZATION_Master_Account_Master '
 'PMAM with (nolock)   inner join \r\n'
 '\t\t\t\t\t\tFINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL k '
 'with(nolock) \r\n'
 '\t\t\t\t\t\ton PMAM.ORGANIZATION_Master_Id =k.organization_master_id\r\n'
 '\t\t\t\t\t\tINNER JOIN [FINCORE_DB].[Fincore].Account_Master AM with '
 '(nolock)         \r\n'
 '\t\t\t\t\t\tON PMAM.Account_Master_Id = AM.Account_Master_Id          \r\n'
 '\t\t\t\t\t\tWHERE PMAM.Is_Active_Flag = 1    and AM.Acc

Done
('\r\n'
 '       MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES  \r\n'
 '\t   WITH (HOLDLOCK) as target USING (       \r\n'
 '\t   SELECT   PMCFR.ORGANIZATION_Master_Id,     \r\n'
 '\t   SUM( 1 ) AS CBWTR_COUNT            \r\n'
 '\t   FROM [FINCORE_DB].[Fincore].ORGANIZATION_Master_Cluster_Fingate_Report '
 'PMCFR  with (nolock) inner join \r\n'
 '\t   FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_INCREMENTAL k '
 'with(nolock) on PMCFR.ORGANIZATION_Master_Id=k.organization_master_id\r\n'
 '\t   INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock) ON '
 'PMCFR.Report_Type_Id = B.id_    \r\n'
 '\t   inner join FINCORE_DB.Fincore_Analytics.RISK_REPORT rr with (nolock) on '
 'PMCFR.Batch_Id=rr.BATCH_ID and PMCFR.Report_Id=rr.REPORT_ID     \r\n'
 "\t   WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CBWTR' and "
 'rr.RISK_SCORE<5 GROUP BY PMCFR.ORGANIZATION_Master_Id           )  \r\n'
 '\t   as source   ON target.ORGANIZATION_MASTER

 '\t\t\t\t\t\t\t\t\t\t  \r\n'
 '\t\t\t\t\t\t\t\t\t\t  WHERE A.EMAIL_ID IS NOT NULL     \r\n'
 '\t\t\t\t\t\t\t\t\t\t  and A.status_active_flag = 1 and    B.is_active_flag '
 '=1   GROUP BY B.ORGANIZATION_Master_Id  )      \r\n'
 '\t\t\t\t\t\t\t\t\t\t  AS source   ON    target.ORGANIZATION_MASTER_ID = '
 'source.ORGANIZATION_master_id        \r\n'
 "\t\t\t\t\t\t\t\t\t\t  and RULE_NO= 'O002001009'     WHEN "
 'MATCHED              \r\n'
 '\t\t\t\t\t\t\t\t\t\t  THEN UPDATE SET '
 'target.RULE_VALUE=source.no_of_Email_Ids        \r\n'
 '\t\t\t\t\t\t\t\t\t\t  WHEN NOT MATCHED BY TARGET                   \r\n'
 '\t\t\t\t\t\t\t\t\t\t  THEN INSERT '
 '(ORGANIZATION_MASTER_ID,RULE_NO,RULE_VALUE,CREATE_DATE)    \r\n'
 '\t\t\t\t\t\t\t\t\t\t  VALUES '
 "(source.ORGANIZATION_master_id,'O002001009',source.no_of_Email_Ids "
 ',getdate()); ')
Done
('MERGE INTO '
 '[FINCORE_DB].[FINCORE_ANALYTICS].[RISK_ORGANIZATION_SUMMARY_STG_RULES] AS '
 'TARGET USING (  \r\n'
 'SELECT W.ORGANIZATION_MASTER_ID, \r\n'

Done
('MERGE INTO '
 'FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES             '
 'AS TARGET USING (       \r\n'
 'SELECT TEMP.ORGANIZATION_MASTER_ID ,  \r\n'
 'CAST(((TEMP.CAPITAL_INVESTMENTS-TEMP.CBDT_INCOME_RANGE)/TEMP.CBDT_INCOME_RANGE)*100 '
 'AS DECIMAL (16,2))AS  CAPITAL_INVESTMENT_PERCENTAGE       FROM  ( \r\n'
 'SELECT   RPSS.ORGANIZATION_MASTER_ID ,  \r\n'
 'CBDT_INCOME_RANGE ,     \r\n'
 'SUM(CDSL.VALUATION) AS CAPITAL_INVESTMENTS   \r\n'
 'FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_INCREMENTAL K '
 'WITH(NOLOCK)     \r\n'
 'INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG RPSS '
 'WITH(NOLOCK)   \r\n'
 'ON K.ORGANIZATION_MASTER_ID = RPSS.ORGANIZATION_MASTER_ID \r\n'
 'INNER JOIN EXTL_DB.CDSL.CDSL_RESPONSE_OUTPUT CDSL WITH (NOLOCK)       \r\n'
 'ON  RPSS.PAN=CDSL.PAN   \r\n'
 'WHERE DATEDIFF(DAY,CDSL.VALUATION_DATE,GETDATE())<=365 AND '
 'RPSS.CBDT_INCOME_RANGE IS NOT NULL   AND RPSS.CBDT_INCOME_RANGE!=0 \r\n'
 'GROUP BY RPSS

IntegrityError: ('23000', "[23000] [FreeTDS][SQL Server]Violation of PRIMARY KEY constraint 'XPK_RISK_ORGANIZATION_SUMMARY_STG_RULES'. Cannot insert duplicate key in object 'Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES'. The duplicate key value is (104, O003001002, FOODS AND BEVERAGES). (2627) (SQLExecDirectW)")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



(23000, [23000] [FreeTDS][SQL Server]Violation of PRIMARY KEY constraint XPK_RISK_ORGANIZATION_SUMMARY_STG_RULES. Cannot insert duplicate key in object Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES. The duplicate key value is (104, O003001002, FOODS AND BEVERAGES). (2627) (SQLExecDirectW))
update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set ERROR_DESC = '(23000, [23000] [FreeTDS][SQL Server]Violation of PRIMARY KEY constraint XPK_RISK_ORGANIZATION_SUMMARY_STG_RULES. Cannot insert duplicate key in object Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES. The duplicate key value is (104, O003001002, FOODS AND BEVERAGES). (2627) (SQLExecDirectW))'  where  JOB_RUN_ID = 999
Traceback (most recent call last):
  File "<ipython-input-18-2113c302be76>", line 26, in <module>
    get_ipython().run_line_magic('run', 'Organization_Risk/Master_data_pop_all_data_ORG.ipynb')
  File "/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2326, in run_line_magic
    r

TypeError: object of type 'NoneType' has no len()

In [15]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [16]:
# try:

#     conn.autocommit = False
#     sql_delete_query1  = "delete from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION"
#     sql_delete_query2  = "delete from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_ANALYTICAL_RULES"
#     sql_delete_query3  = "delete from FINCORE_DB.Fincore_Analytics.RISK_LKP_ENTITY_THRESHOLD where RULE_NO like 'O0%'"
#     sql_delete_query4  = "delete from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES"
#     sql_delete_query5  = "delete from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG"

#     cursor.execute(sql_delete_query1)
#     cursor.execute(sql_delete_query2)
#     cursor.execute(sql_delete_query3)
#     cursor.execute(sql_delete_query4)
#     cursor.execute(sql_delete_query5)
    
# #     cursor.commit()
    
#     one = pd.read_sql_query("select * from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG with (nolock)",conn)
#     print(one)
    

#     %run Master_data_pop_all_data_ORG.ipynb
#     print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#     %run Entity_Risk_V2_ORG.ipynb
#     print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
#     %run Threshold_Computation_V2_ORG.ipynb
#     print("###############################################################################################")
#     %run Entity_Risk_Calculation_V2_1_ORG.ipynb
#     print("**************************************************************************************************")
# #     cursor.execute(sql_select_query1)
# #     conn.commit()


        
        
# except Exception as error:
#     %store -r job_id
#     error = str(error)
#     error = error.replace('\'','')
#     error = error.replace('\"','')
#     print("Failed")
#     conn.rollback()
#     print("rolled back")
#     conn.execute(f'''update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set ERROR_DESC = '{error}'  where  JOB_RUN_ID = {job_id}''')
#     conn.commit()
#     conn.close()

    

    
    
    
    


In [17]:
# %run master_data_pop_incremental.ipynb
# %run Entity_Risk_V2_Incremental_data.ipynb
# %run Threshold_Computation_V2_Incremental_data.ipynb
# %run Entity_Risk_Calculation_V2_1_Incremental_data.ipynb

In [18]:
# conn.close()

In [ ]:
# try:
#     %run master_data_pop_all_data.ipynb
#     %run Entity_Risk_V2.ipynb
#     %run Threshold_Computation_V2.ipynb
#     %run Entity_Risk_Calculation_V2_1.ipynb
# except Exception as error:
#     %store -r job_id
#     error = str(error)
#     error = error.replace('\'','')
#     error = error.replace('\"','')
#     print(error)
#     conn.execute(f'''update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set ERROR_DESC = '{error}'  where  JOB_RUN_ID = {job_id}''')
#     conn.commit()
#     conn.close()

In [ ]:
# try:
#     %run master_data_pop.ipynb
#     %run Entity_Risk_V2_Incremental_data.ipynb
#     %run Threshold_Computation_V2_Incremental_data.ipynb
#     %run Entity_Risk_Calculation_V2_1_Incremental_data.ipynb
# except Exception as error:
    
#     %store -r job_id
#     error = str(error)
#     error = error.replace('\'','')
#     error = error.replace('\"','')
#     conn.execute(f'''update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set ERROR_DESC = '{error}'  where  JOB_RUN_ID = {job_id}''')
#     conn.commit()
#     conn.close()

In [ ]:
# %run Master_data_pop_all_data_ORG.ipynb
# print("A1")
# %run Entity_Risk_V2_ORG.ipynb
# print("A2")
# %run Threshold_Computation_V2_ORG.ipynb
# print("A3")
# %run Entity_Risk_Calculation_V2_1_ORG.ipynb
# print("A4")